In [20]:
import tensorflow as tf
import struct
from tensorflow.core.example import example_pb2
from collections import Counter
from unidecode import unidecode
import nltk
import json

In [10]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

def remove_non_ascii(text):
    try:
        return unidecode(unicode(text, encoding = "utf-8"))
    except:
        return unidecode(str(text))

def abstract2sents(abstract):
  """Splits abstract text from datafile into list of sentences.

  Args:
    abstract: string containing <s> and </s> tags for starts and ends of sentences

  Returns:
    sents: List of sentence strings (no tags)"""
  SENTENCE_START = '<s>'
  SENTENCE_END = '</s>'
  try:
    abstract = abstract.decode(encoding="utf-8", errors="strict")
  except:
    pass
  cur = 0
  sents = []
  while True:
    try:
      start_p = abstract.index(SENTENCE_START,cur)
      end_p = abstract.index(SENTENCE_END, start_p + 1)
      cur = end_p + len(SENTENCE_END)
      sents.append(abstract[start_p+len(SENTENCE_START):end_p].strip())
    except ValueError as e: # no more sentences
      return sents


In [24]:
mode = 'val'
json_dump = []
reader = open('/home/yaserkl/data/cnn_dm/finished_files/{}.bin'.format(mode), 'rb')
cnt = 0
while True:
    len_bytes = reader.read(8)
    if not len_bytes: break # finished reading this file
    str_len = struct.unpack('q', len_bytes)[0]
    example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
    
    e = example_pb2.Example.FromString(example_str)
    article_text = remove_non_ascii(e.features.feature['article'].bytes_list.value[0]) # the article text was saved under the key 'article' in the data files
    abstract_text = remove_non_ascii(e.features.feature['abstract'].bytes_list.value[0]) # the abstract text was saved under the key 'abstract' in the data files
    if len(article_text)==0 or len(abstract_text)==0: # See https://github.com/abisee/pointer-generator/issues/1
        continue

    article_sents = sent_detector.tokenize(remove_non_ascii(article_text))
    abstract_sents = abstract2sents(remove_non_ascii(abstract_text))
    _art = [_.split() for _ in article_sents]
    _abs = [_.split() for _ in abstract_sents]
    json_dump.append({'src':_art, 'tgt':_abs})
    cnt +=1

In [25]:
with open('/home/yaserkl/data/bertsum/json_data/{}.json'.format(mode), 'w') as fw:
    fw.write(json.dumps(json_dump, indent=4))